In [1]:
import glob, os, json
import time
import pickle
import solver
import traceback

In [14]:
tspLibPath = "../data/tsplib/"

# Load TSP instances
tspLibInstances = []
tspInstanceNames = []
for file in glob.glob(tspLibPath + "*.tsp"):
    try:
        tsp = solver.loadTSPLib(file)
        tspLibInstances.append(tsp)
        tspInstanceNames.append(os.path.basename(file))
    except:
        traceback.print_exc()

In [15]:
generatedPath = "../data/generated/"

# Load TSP instances
generatedInstances = []
generatedInstanceNames = []
for file in glob.glob(generatedPath + "*.pytsp"):
    try:
        tspFile = open(file, "rb")
        tsp = pickle.load(tspFile)
        generatedInstances.append(tsp)
        generatedInstanceNames.append(os.path.basename(file))
        tspFile.close()
    except:
        traceback.print_exc()

In [16]:
allInstances = tspLibInstances + generatedInstances
allInstanceNames = tspInstanceNames + generatedInstanceNames

In [2]:
def measureFeature(name, deterministic, tsp, data, features, featureFunction, storeResult=True):
    start = time.time()
    value = featureFunction(tsp, data)
    end = time.time()
    
    storedValue = value
    if type(storedValue).__module__=='numpy': # if value is numpy.*: > to python list
        storedValue = storedValue.tolist()
    
    if storeResult:
        if deterministic:
            features[name] = storedValue
        else:
            valueName = name + "Values"
            if valueName not in features:
                features[valueName] = []

            features[valueName].append(storedValue)
        
    timeName = name + "Times"
    
    if timeName not in features:
        features[timeName] = []
    
    features[timeName].append(end - start)
    
    return value

In [3]:
def stripNumpyDict(dictionary):
    for key in dictionary:
        value = dictionary[key]
        if isinstance(value, dict):
            stripNumpyDict(value)
        if isinstance(value, list):
            dictionary[key] = stripNumpyArray(value)
        elif isinstance(value, numpy.ndarray):
            dictionary[key] = stripNumpyArray(value.tolist())
        elif type(value).__module__=='numpy': # if value is numpy.*: > to python list
            dictionary[key] = value.item()
            
def stripNumpyArray(array):
    newArray = []
    for item in array:
        if isinstance(item, list):
            newArray.append(stripNumpyArray(item))
        elif isinstance(item, numpy.ndarray):
            newArray.append(stripNumpyArray(item.tolist()))
        elif type(item).__module__=='numpy': # if value is numpy.*: > to python list
            newArray.append(item.item())
        else:
            newArray.append(item)
            
    return newArray

In [9]:
from network.features import *
from network.simpleFeatures import *
from network.complexFeatures import *
from network.heuristicFeatures import *
import json

nLowestEdgeCosts = 10
nGeneratedSolutions = 100

def generateSimpleFeatures(tsp, features, n):
    simple = {}
    for i in range(n):
        print("Starting Simple Features %d" % (i + 1))
        measureFeature("numberVertices", True, tsp, None, simple, lambda tsp, _: numberVertices(tsp))
        
        vertexCosts = measureFeature("vertexCostPrep", True, tsp, None, simple, lambda tsp, _: buildVertexCosts(tsp), False)
        data = {"vertexCosts": vertexCosts}
        measureFeature("lowestVertexCost", True, tsp, data, simple, lambda _, data: lowestVertexCost(data["vertexCosts"]))
        measureFeature("highestVertexCost", True, tsp, data, simple, lambda _, data: highestVertexCost(data["vertexCosts"]))
        measureFeature("averageVertexCost", True, tsp, data, simple, lambda _, data: averageVertexCost(data["vertexCosts"]))
        measureFeature("standardDeviationVertexCost", True, tsp, data, simple, lambda _, data: standardDeviationVertexCost(data["vertexCosts"]))
        measureFeature("medianVertexCost", True, tsp, data, simple, lambda _, data: medianVertexCost(data["vertexCosts"]))
        
        measureFeature("sumCostNearestNeighbor", True, tsp, None, simple, lambda tsp, _: sumCostNearestNeighbor(tsp))
        measureFeature("numberEdges", True, tsp, None, simple, lambda tsp, _: numberEdges(tsp))
        measureFeature("lowestEdgeCost", True, tsp, None, simple, lambda tsp, _: lowestEdgeCost(tsp))
        measureFeature("highestEdgeCost", True, tsp, None, simple, lambda tsp, _: highestEdgeCost(tsp))
        measureFeature("averageEdgeCost", True, tsp, None, simple, lambda tsp, _: averageEdgeCost(tsp))
        measureFeature("standardDeviationEdgeCost", True, tsp, None, simple, lambda tsp, _: standardDeviationEdgeCost(tsp))
        measureFeature("medianEdgeCost", True, tsp, None, simple, lambda tsp, _: medianEdgeCost(tsp))
        measureFeature("sumNLowestEdgeCost", True, tsp, None, simple, lambda tsp, _: sumNLowestEdgeCost(tsp, nLowestEdgeCosts))
        
        print("Ending Simple Features %d" % (i + 1))
        
    features["simpleFeatures"] = simple
    
def generateComplexFeatures(tsp, features, n):
    complexFeatures = {}
    for i in range(n):
        print("Starting Complex Features %d" % (i + 1))
        measureFeature("averageShortestPathGeodesicDistance", True, tsp, None, complexFeatures, lambda tsp, _: averageShortestPathGeodesicDistance(tsp))
        measureFeature("averageGeodesicDistance", True, tsp, None, complexFeatures, lambda tsp, _: averageGeodesicDistance(tsp))
        
        globalEfficiencyValue = measureFeature("globalEfficiency", True, tsp, None, complexFeatures, lambda tsp, _: globalEfficiency(tsp))
        data = {"globalEfficiency": globalEfficiencyValue}
        measureFeature("harmonicMeanGeodesicDistance", True, tsp, None, complexFeatures, lambda tsp, _: harmonicMeanGeodesicDistance(tsp))
        
        measureFeature("networkVulnerability", True, tsp, data, complexFeatures, lambda tsp, data: networkVulnerability(tsp, data["globalEfficiency"]))
        
        measureFeature("clusteringCoefficientTransitivity", True, tsp, None, complexFeatures, lambda tsp, _: clusteringCoefficientTransitivity(tsp))
        measureFeature("alternateClusteringCoefficient", True, tsp, None, complexFeatures, lambda tsp, _: alternateClusteringCoefficient(tsp))
        
        connectedEdgeCountsValue = measureFeature("connectedEdgeCountsPrep", True, tsp, None, complexFeatures, lambda tsp, _: connectedEdgeCounts(tsp), False)
        data = {"connectedEdgeCounts": connectedEdgeCountsValue}
        measureFeature("weightedClusteringCoefficient", True, tsp, data, complexFeatures, lambda tsp, data: weightedClusteringCoefficient(tsp, data["connectedEdgeCounts"]))
        measureFeature("networkCyclicCoefficient", True, tsp, data, complexFeatures, lambda tsp, data: networkCyclicCoefficient(tsp, data["connectedEdgeCounts"]))
        measureFeature("maxVertexDegree", True, tsp, data, complexFeatures, lambda _, data: maxVertexDegree(data["connectedEdgeCounts"]))
        measureFeature("edgeDegreeCorrelation", True, tsp, data, complexFeatures, lambda tsp, data: edgeDegreeCorrelation(tsp, data["connectedEdgeCounts"]))
        
        measureFeature("entropyDegreeDistribution", True, tsp, None, complexFeatures, lambda tsp, _: entropyDegreeDistribution(tsp))
        measureFeature("targetEntropy", True, tsp, None, complexFeatures, lambda tsp, _: targetEntropy(tsp))
        measureFeature("vertexParticipationCoefficient", True, tsp, None, complexFeatures, lambda tsp, _: vertexParticipationCoefficient(tsp))
        measureFeature("edgeReciprocity", True, tsp, None, complexFeatures, lambda tsp, _: edgeReciprocity(tsp))
        measureFeature("adjacencyCorrelationCoefficient", True, tsp, None, complexFeatures, lambda tsp, _: adjacencyCorrelationCoefficient(tsp))
        
        print("Ending Complex Features %d" % (i + 1))
        
    features["complexFeatures"] = complexFeatures
    
def generateHeuristicFeatures(tsp, features, n):
    heuristicFeatures = {}
    for i in range(n):
        print("Starting Heuristic Features %d" % (i + 1))
        
        random = measureFeature("randomSolutionsPrep", False, tsp, None, heuristicFeatures, lambda tsp, _: randomSolutions(tsp, nGeneratedSolutions), False)
        data = {"randomSolutions": random}
        randomNeighbor = measureFeature("randomNeighborSolutionsPrep", False, tsp, data, heuristicFeatures, lambda tsp, data: list(map(lambda solution: neighborSolutions(tsp, solution, nGeneratedSolutions), data["randomSolutions"])), False)
        
        # Remove most of randomNeighbors from features
        # heuristicFeatures["randomNeighborSolutionsPrep"] = heuristicFeatures["randomNeighborSolutionsPrep"][:min(5, nGeneratedSolutions)]
        
        data = {"randomSolutions": random, "randomNeighborSolutions": randomNeighbor}
        measureFeature("proportionsNeighborsWithBetterSolution", False, tsp, data, heuristicFeatures, lambda tsp, data: proportionsNeighborsWithBetterSolution(tsp, data["randomSolutions"], data["randomNeighborSolutions"]))
        measureFeature("averageRatioNeighborsWithBetterSolution", False, tsp, data, heuristicFeatures, lambda tsp, data: averageRatioNeighborsWithBetterSolution(tsp, data["randomSolutions"], data["randomNeighborSolutions"]))
        measureFeature("randomNeighborSolutionQuality", False, tsp, data, heuristicFeatures, lambda tsp, data: randomNeighborSolutionQuality(tsp, data["randomSolutions"], data["randomNeighborSolutions"]))
        
        greedy = measureFeature("greedySolutionsPrep", False, tsp, None, heuristicFeatures, lambda tsp, _: greedySolutions(tsp, nGeneratedSolutions), False)
        data = {"randomSolutions": random, "greedySolutions": greedy}
        measureFeature("greedySolutionQuality", False, tsp, data, heuristicFeatures, lambda tsp, data: greedySolutionQuality(tsp, data["randomSolutions"], data["greedySolutions"]))
        measureFeature("averageRatioGreedySolution", False, tsp, data, heuristicFeatures, lambda tsp, data: averageRatioGreedySolution(tsp, data["randomSolutions"], data["greedySolutions"]))
        
        parents = measureFeature("parentSolutionPairsPrep", False, tsp, data, heuristicFeatures, lambda tsp, data: solutionPairs(tsp, data["randomSolutions"], nGeneratedSolutions), False)
        data = {"parentSolutionPairs": parents}
        children = measureFeature("childrenSolutionPairsPrep", False, tsp, data, heuristicFeatures, lambda tsp, data: childrenSolutionPairs(tsp, data["parentSolutionPairs"]), False)
        
        data = {"parentSolutionPairs": parents, "childrenSolutionPairs": children}
        measureFeature("proportionOffspringBetterSolution", False, tsp, data, heuristicFeatures, lambda tsp, data: proportionOffspringBetterSolution(tsp, data["parentSolutionPairs"], data["childrenSolutionPairs"]))
        measureFeature("averageRatioOffspringBetterSolution", False, tsp, data, heuristicFeatures, lambda tsp, data: averageRatioOffspringBetterSolution(tsp, data["parentSolutionPairs"], data["childrenSolutionPairs"]))
        measureFeature("averageTimesOffspringBetterSolution", False, tsp, data, heuristicFeatures, lambda tsp, data: averageTimesOffspringBetterSolution(tsp, data["parentSolutionPairs"], data["childrenSolutionPairs"]))
        
        data = {"greedySolutions": greedy}
        greedyEdges = measureFeature("greedyEdgesPrep", False, tsp, data, heuristicFeatures, lambda tsp, data: edges(tsp, data["greedySolutions"]), False)
        data = {"greedyEdges": greedyEdges}
        measureFeature("averageSharedEdges", False, tsp, data, heuristicFeatures, lambda tsp, data: averageSharedEdges(tsp, data["greedyEdges"]))
        measureFeature("relativeFrequencyCommonGreedyEdge", False, tsp, data, heuristicFeatures, lambda tsp, data: relativeFrequencyCommonGreedyEdge(tsp, data["greedyEdges"]))
        
        print("Ending Heuristic Features %d" % (i + 1))
        
    features["heuristicFeatures"] = heuristicFeatures
        
def generateFeatures(tsp, filename, n):
    features = {}
    
    generateSimpleFeatures(tsp, features, n)
    generateComplexFeatures(tsp, features, n)
    generateHeuristicFeatures(tsp, features, n)
    
    stripNumpyDict(features)
    
    print("Writing File %s" % (filename))
    
    with open(filename, "w") as file:
        json.dump(features, file, indent="\t")
        
    print("Finished writting")


In [41]:
allInstanceNames

NameError: name 'allInstanceNames' is not defined

In [5]:
tsp = solver.loadTSPLib("/Users/adam/Documents/Rose Repos/Thesis/data/tsplib/ch130.tsp")

In [ ]:
features = {}
generateFeatures(tsp, "../data/features/tsplib/ch130.json", 5)
# generateHeuristicFeatures(tsp, features, 1)

Starting Simple Features 1
Ending Simple Features 1
Starting Simple Features 2
Ending Simple Features 2
Starting Simple Features 3
Ending Simple Features 3
Starting Simple Features 4
Ending Simple Features 4
Starting Simple Features 5
Ending Simple Features 5
Starting Complex Features 1
Ending Complex Features 1
Starting Complex Features 2
Ending Complex Features 2
Starting Complex Features 3
Ending Complex Features 3
Starting Complex Features 4
Ending Complex Features 4
Starting Complex Features 5
Ending Complex Features 5
Starting Heuristic Features 1
Ending Heuristic Features 1
Starting Heuristic Features 2
Ending Heuristic Features 2
Starting Heuristic Features 3
Ending Heuristic Features 3
Starting Heuristic Features 4
Ending Heuristic Features 4
Starting Heuristic Features 5


In [7]:
stripNumpyDict(features)

with open("test.json", "w") as file:
    json.dump(features, file, indent="\t")